# TutQ4 Quantum General Adverserial Network

This demo constructs a General Adverserial Network (GAN)
from a quantum circuit that serves as a generator and a second
quantum circuit which takes the role of a discriminator.

Quantum GANs could find application in the task of "unitary learning", which means to learn a quantum circuit that prepares a state given for example by a device with unknown dynamics.

### Imports

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)
dev = qml.device('default.qubit', wires=3)

### Classical and quantum nodes

Instead of drawing samples from a "true" distribution and the generator, the idea of the quantum GAN is to have a "true" circuit and a generator circuit prepare a quantum state, on which the circuit of the discriminator has to act as a classifier.

The "true circuit" is a simple rotation.

In [2]:
def true(phi, theta, omega):
    qml.Rot(phi, theta, omega, wires=0)

The generator and discriminator circuits are made up of the same basic routine.

In [3]:
def routine(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)


def generator(w):
    routine(w)


def discriminator(w):
    routine(w)

This is how the generator and discriminator models are built:

In [4]:
@qml.qnode(dev)
def true_disc_circuit(phi, theta, omega, disc_weights):
    true(phi, theta, omega)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval.PauliZ(2)

### Cost

The cost is associated with the probability that the discriminator guesses "true" for a "fake" state (i.e. a state prepared by the generator instead of the discriminator).

In [5]:
def prob_real_true(disc_weights):
    true_disc_output = true_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true

def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true # want to minimize this prob

def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights) 
    return cost

def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

### Optimization

We initialize the fixed angles of the "true circuit", as well as some variables.

In [6]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7

gen_weights = np.array([0] + [np.pi] + [0] * 7) 
disc_weights = np.random.normal(size=[9])

Creating an optimizer...

In [7]:
opt = qml.GradientDescentOptimizer(0.1)

..we first train the discriminator.

In [ ]:
for it in range(50):
    disc_weights = opt.step(disc_cost, disc_weights) 
    cost = disc_cost(disc_weights)
    print("Step {}: cost = {}".format(it+1, cost))

Step 1: cost = 0.0
Step 2: cost = 0.0
Step 3: cost = 0.0
Step 4: cost = 0.0
Step 5: cost = 0.0
Step 6: cost = 0.0
Step 7: cost = 0.0
Step 8: cost = 0.0
Step 9: cost = 0.0
Step 10: cost = 0.0
Step 11: cost = 0.0
Step 12: cost = 0.0
Step 13: cost = 0.0
Step 14: cost = 0.0
Step 15: cost = 0.0
Step 16: cost = 0.0
Step 17: cost = 0.0
Step 18: cost = 0.0
Step 19: cost = 0.0
Step 20: cost = 0.0
Step 21: cost = 0.0
Step 22: cost = 0.0
Step 23: cost = 0.0
Step 24: cost = 0.0
Step 25: cost = 0.0
Step 26: cost = 0.0
Step 27: cost = 0.0
Step 28: cost = 0.0
Step 29: cost = 0.0
Step 30: cost = 0.0
Step 31: cost = 0.0
Step 32: cost = 0.0
Step 33: cost = 0.0
Step 34: cost = 0.0


The probability of the discriminator to be correct should be close to one at the discriminators optimum.

In [ ]:
prob_real_true(disc_weights)

Vice versa, the probability of the discriminator to be wrong should be close to zero at the discriminators optimum.

In [ ]:
prob_fake_true(gen_weights, disc_weights)

In the adverserial game we have to now train the generator (and one can continue training the models in an alternating fashion).

In [ ]:
for it in range(100):
    gen_weights = opt.step(gen_cost, gen_weights)
    cost = -gen_cost(gen_weights)
    if it % 5 == 0:
        print("Step {}: cost = {}".format(it+1, cost))

At the optimum of the generator, the probabilities should be swapped, in other words, the probability of the discriminator to be fooled should be close to 1.

In [ ]:
# should be close to one at G's optimum
prob_real_true(disc_weights)

At the joint optimum the cost will be close to zero.

In [ ]:
 # should be close to zero at joint optimum
disc_cost(disc_weights)